In [1]:
import pandas as pd
import re
import numpy as np

## Clean and format gradutation rates

In [11]:
columns = ['LEAID', 'STNAM', 'FIPST', 'LEANM', 'ALL_RATE_1718', 'ALL_COHORT_1718']
df = pd.read_csv('../Data/Education/Graduation_Rates/graduation_rates.csv', usecols=columns)

In [12]:
mask = [ (i in '123456789' and int(i) < 50) for i in df['ALL_RATE_1718']]
mask2 = ['#' in i for i in df['LEANM']]
mask3 = [re.match(r'[0-9][0-9][\-][0-9][0-9]', i ) != None for i in df['ALL_RATE_1718']]

In [13]:
def clean_rate(i):
    def avg_rate(i):
        return (int(i[3:5]) + int(i[0:2])) /2 
    
    reg_range = r'[0-9][0-9][\-][0-9][0-9]'
    reg_less_greater = '^[L|G]'
    reg_other = r'^[P]'
    try:
        if re.match(reg_range, str(i)) != None:
            return str(avg_rate(i))

        if re.match(reg_less_greater, str(i)) !=None:
            return i[2:]

        if re.match(reg_other, str(i)) or len(str(i)) > 3:
            return np.nan
        
        return i
    except Exception as e:
        print('Exception caught: ', e, i)
        

In [14]:
df.ALL_RATE_1718 = pd.Series([clean_rate(i) for i in df['ALL_RATE_1718']])

In [15]:
df = df.dropna()
df

,STNAM,FIPST,LEAID,LEANM,ALL_COHORT_1718,ALL_RATE_1718
0,ALABAMA,1,100005,Albertville City,312,94
1,ALABAMA,1,100006,Marshall County,416,91
2,ALABAMA,1,100007,Hoover City,1120,94
3,ALABAMA,1,100008,Madison City,923,96
4,ALABAMA,1,100011,Leeds City,128,95
...,...,...,...,...,...,...
12307,WYOMING,56,5605762,Sweetwater County School District #2,209,92
12308,WYOMING,56,5605820,Washakie County School District #2,8,50
12309,WYOMING,56,5605830,Teton County School District #1,188,87.0
12310,WYOMING,56,5606090,Weston County School District #7,22,80


## Map LEAID to counties

In [30]:
columns = ['LEAID', 'STATE', 'CNTY', 'NMCNTY']
df_map = pd.read_csv('../Data/Education/Graduation_Rates/Public_School_Locations.csv', usecols=columns).drop_duplicates()
df_map

,LEAID,STATE,CNTY,NMCNTY
0,100005,AL,1095,Marshall County
6,100006,AL,1095,Marshall County
21,100007,AL,1073,Jefferson County
22,100007,AL,1117,Shelby County
38,100008,AL,1089,Madison County
...,...,...,...,...
101876,7200030,PR,72139,Trujillo Alto Municipio
101934,7200030,PR,72017,Barceloneta Municipio
102148,7800002,VI,78010,St. Croix Island
102162,7800030,VI,78030,St. Thomas Island


In [33]:
df_merged = pd.merge(df, df_map, on='LEAID')
df_merged.shape

(13028, 9)

In [35]:
df_merged.to_csv('../Data/Cleaned/Eduction_Graduation_Rates.csv')